In [1]:
import pandas as pd
import numpy as np
import re
import emoji
import nltk

In [2]:
tweets = pd.read_csv('D:\\Github\\financial_dashboard\data_scrappers\\tweets.csv', index_col=0)
# wsb_posts = pd.read_csv('D:\\Github\\financial_dashboard\data_scrappers\\wsb_posts.csv')
wsb_titles = pd.read_csv('D:\\Github\\financial_dashboard\data_scrappers\\wsb_title.csv')

## Twitter

In [3]:
tweets.columns = ['Date','Ticker','Tweet']

In [4]:
# nltk.download('words')
# words = set(nltk.corpus.words.words())

def cleaner(tweet):
    tweet = re.sub(r"RT @[\w]*:","",tweet)
    tweet = re.sub("@[A-Za-z0-9]+","",tweet) #Remove @ sign
    tweet = re.sub(r"(?:\@|http?\://|https?\://|www)\S+", "", tweet) #Remove http links
    tweet = re.sub(r"[^a-zA-Z\s]","",tweet)
    tweet = " ".join(tweet.split())
    tweet = ''.join(c for c in tweet if c not in emoji.UNICODE_EMOJI) #Remove Emojis
    tweet = tweet.replace("#", "").replace("_", " ").replace('*','').replace('$','') #Remove hashtag sign but keep the text
#     tweet = " ".join(w for w in nltk.wordpunct_tokenize(tweet) \
#          if w.lower() in words or not w.isalpha())
    return tweet

# https://stackoverflow.com/questions/64719706/cleaning-twitter-data-pandas-python

In [5]:
tweets['Tweet'] = tweets['Tweet'].map(lambda x: cleaner(x))

In [8]:
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

analyzer = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\jacob\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


## Reddit 

In [ ]:
wsb_titles['GME'].map(lambda x: cleaner(x))

In [ ]:
for col in wsb_titles.columns:
    wsb_titles[col] = wsb_titles[col].map(lambda x: np.nan if pd.isna(x)  else cleaner(x))


## Model

In [16]:
analyzer = SentimentIntensityAnalyzer()
# [analyzer.polarity_scores(x)['compound'] for x in tweets['Tweet'][:10]] 

def sentiment_df(df,col):

    df['compund'] = [analyzer.polarity_scores(x)['compound'] for x in df[col]]
    df['neg'] = [analyzer.polarity_scores(x)['neg'] for x in df[col]] 
    df['neu'] = [analyzer.polarity_scores(x)['neu'] for x in df[col]] 
    df['pos'] = [analyzer.polarity_scores(x)['pos'] for x in df[col]] 

    return df.drop(columns = col)

    
    
    
    

In [17]:
sentiment_df(tweets.head(),'Tweet')

<ipython-input-16-f581433b1c2a>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['compund'] = [analyzer.polarity_scores(x)['compound'] for x in df[col]]
<ipython-input-16-f581433b1c2a>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['neg'] = [analyzer.polarity_scores(x)['neg'] for x in df[col]]
<ipython-input-16-f581433b1c2a>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the do

,Date,Ticker,compund,neg,neu,pos
0,2021-03-30 19:10:08,GME,0.5719,0.000,0.802,0.198
1,2021-03-30 12:45:00,GME,0.5106,0.000,0.699,0.301
2,2021-03-29 23:37:36,GME,-0.5574,0.259,0.564,0.177
3,2021-04-01 05:37:05,GME,0.0000,0.000,1.000,0.000
4,2021-04-01 05:36:52,GME,0.7269,0.000,0.711,0.289


In [ ]:
pd.melt(wsb_titles, var_name='Ticker', value_name='Title').dropna() ## melts all the columns into one column and adds ticker column

In [19]:
tweets.dropna().shape 

(193729, 3)